In [ ]:
!pip install langchain langchain-core langchain-unstructured langchain-community langchain-mistralai
!pip install unstructured unstructured[all-docs] bitsandbytes
!pip install faiss-cpu
!apt-get update && apt-get install -y poppler-utils tesseract-ocr

In [ ]:
import os
import getpass

import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

if not os.getenv("UNSTRUCTURED_API_KEY"):
    os.environ["UNSTRUCTURED_API_KEY"] = getpass.getpass("UNSTRUCTURED_API_KEY")

if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("HF_TOKEN")

print("Unstructured API Key installed:", os.getenv("UNSTRUCTURED_API_KEY") is not None)
print("HF API Key installed:", os.getenv("HF_TOKEN") is not None)

In [ ]:
file_path = "/content/175_2024_ND-CP_609382.pdf"

all_chunks = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

try:
    print(f"Đang xử lý: {os.path.basename(file_path)}")
    loader = UnstructuredPDFLoader(file_path, mode="elements", strategy="fast")
    documents = loader.load()
    chunks = text_splitter.split_documents(documents)
    all_chunks.extend(chunks)
    print(f"Số lượng documents đã tải: {len(documents)}")
    print(f"Số lượng chunks đã tạo: {len(chunks)}")
    del documents
    del chunks
except Exception as e:
    print(f"Lỗi khi đang xử lý {file_path}: {e}")


unique_text = {}
chunk_processed_unique = []
for chunk in all_chunks:
    if chunk.page_content not in unique_text:
        unique_text[chunk.page_content] = 1
        chunk_processed_unique.append(chunk)

del unique_text
print(f"Tổng số chunk sau khi loại bỏ trùng lặp: {len(chunk_processed_unique)}")

In [ ]:
model_name = "BAAI/bge-m3"
model_kwargs = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'trust_remote_code': True
}

encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = FAISS.from_documents(chunk_processed_unique, embeddings)
vector_store.save_local("faiss_pdf")

In [ ]:
model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

pipeline = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      max_new_tokens=512,
      temperature=0.65,
      top_p=0.9,
      repetition_penalty=1.03,
      do_sample=True
)

llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
def prompt_vn():
    template = """<s>[INST] <<SYS>>
Bạn là trợ lý AI tiếng Việt hữu ích. Hãy sử dụng ngữ cảnh sau đây để trả lời câu hỏi của người dùng bằng tiếng Việt.
Ngữ cảnh: {context}
<</SYS>>

Câu hỏi: {question}
Trả lời: [/INST]
"""

    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )
    return prompt

prompt = prompt_vn()

retriever = vector_store.as_retriever(
      search_kwargs={"k": 3},
      search_type="similarity",
      distance_strategy=DistanceStrategy.COSINE
)

qa_chain = RetrievalQA.from_chain_type(
      llm,
      retriever=retriever,
      chain_type="stuff",
      chain_type_kwargs={"prompt": prompt},
)

In [ ]:
cau_hoi_mau = [
    "Thời gian thẩm định báo cáo kinh tế kỹ thuật?",
    "Tóm tắt nội dung chính Nghị định 175 về báo cáo tiền khả thi"
]

for cau_hoi in cau_hoi_mau:
    print(f"\nCâu hỏi: {cau_hoi}")
    tra_loi = qa_chain.run(cau_hoi)
    print(f"Trả lời: {tra_loi}")

In [ ]:
def tra_loi_cau_hoi(cau_hoi):
    tra_loi = qa_chain.run(cau_hoi)
    print(f"Câu hỏi: {cau_hoi}")
    print(f"Trả lời: {tra_loi}")

while True:
    cau_hoi_nguoi_dung = input("Nhập câu hỏi (hoặc 'thoat' để dừng): ")
    if cau_hoi_nguoi_dung.lower() == "thoat":
        break
    tra_loi_cau_hoi(cau_hoi_nguoi_dung)